# Doordash Delivery Analysis

Task:
1. Create an Unique Key for each delivery (Index column)
2. Fix 'created_at' & 'actual_delivery_time' data types
3. Parse Date/Time from created_at
4. Parse Date/Time from actual_delivery_time
5. Create delivery_time: actual_time - created_time
6. Reformat dollar amounts: columns=['subtotal', 'min_item_price', 'max_item_price', ]
6. Visualize average delivery time for each market
7. Visualize average delivery time for each order_protocol

Dashboard Ideas:  
Store Performance (Order_protocol performance, total sales, average delivery time, slowest delivery time, fastest delivery time)
Market Performance (""""")
Protocol Performance (""""")
Estimated Time vs Actual Delivery Time analysis

Questions: (All questions asking about all time totals & Possible Trends over time)
1. Which market has the most/least deliveries?
    - Which stores/store categories get the most/least deliveries?
2. Which market has the quickest/slowest deliviries on average?
2. Which order protocol has the most/least deliveries?
3. Which order protocol has the quickest/slowest deliveries on average?
4. What does the average delivery time look like for each market look like daily/weekly/monthly/quarterly/annually
5. What does the average delivery time look like for each order_protocol look like daily/weekly/monthly/quartly/annually

## Data Ingestion

In [580]:
import pandas as pd
from datetime import datetime

In [581]:
data = pd.read_csv("https://raw.githubusercontent.com/ShaneJett-DS/Doordash_Delivery_Analysis/main/delivery_data.csv")
data.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0


In [582]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 16 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   market_id                                     196441 non-null  float64
 1   created_at                                    197428 non-null  object 
 2   actual_delivery_time                          197421 non-null  object 
 3   store_id                                      197428 non-null  int64  
 4   store_primary_category                        192668 non-null  object 
 5   order_protocol                                196433 non-null  float64
 6   total_items                                   197428 non-null  int64  
 7   subtotal                                      197428 non-null  int64  
 8   num_distinct_items                            197428 non-null  int64  
 9   min_item_price                                19

## Data Cleaning

### Create An Index Column

In [583]:
data['delivery_id'] = data.index
data = data[['delivery_id', 'market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration']].copy()
data

,delivery_id,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0
197424,197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0
197425,197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0
197426,197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0


In [584]:
data.columns

Index(['delivery_id', 'market_id', 'created_at', 'actual_delivery_time',
       'store_id', 'store_primary_category', 'order_protocol', 'total_items',
       'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration'],
      dtype='object')

### Parse Dates & Times from Datetime columns

In [585]:


# Convert columns to datetime objects
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])

# Parse out delivery created at info
data['created_at_date'] = pd.to_datetime(data['created_at'])
data['created_at_time'] = data['created_at'].dt.time

# Parse out delivery delivered at info
data['delivered_at_date'] = pd.to_datetime(data['actual_delivery_time'])
data['delivered_at_time'] = data['actual_delivery_time'].dt.time

# Create delivery time columns
data['total_delivery_time'] = (data['actual_delivery_time'] - data['created_at'])
data['total_delivery_seconds'] = data['total_delivery_time'].apply(lambda td: td.total_seconds())
data['total_delivery_minutes'] = (data['total_delivery_seconds'] / 60).round(2)

#data['total_delivery_time'] = data['total_delivery_time'].astype('str').split(' ')[1]

data = data[['delivery_id', 'market_id', 'created_at_date', 'delivered_at_date', 'created_at_time',  'delivered_at_time', 
             'total_delivery_time', 'total_delivery_seconds', 'total_delivery_minutes', 'store_id', 'store_primary_category', 'order_protocol', 
             'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers', 'total_busy_dashers', 
             'total_outstanding_orders', 'estimated_order_place_duration', 'estimated_store_to_consumer_driving_duration']].copy()



data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 197428 entries, 0 to 197427
Data columns (total 22 columns):
 #   Column                                        Non-Null Count   Dtype          
---  ------                                        --------------   -----          
 0   delivery_id                                   197428 non-null  int64          
 1   market_id                                     196441 non-null  float64        
 2   created_at_date                               197428 non-null  datetime64[ns] 
 3   delivered_at_date                             197421 non-null  datetime64[ns] 
 4   created_at_time                               197428 non-null  object         
 5   delivered_at_time                             197421 non-null  object         
 6   total_delivery_time                           197421 non-null  timedelta64[ns]
 7   total_delivery_seconds                        197421 non-null  float64        
 8   total_delivery_minutes                      

### Format Dollar Amounts

In [586]:
data['subtotal'] = data['subtotal'] * .01
data['min_item_price'] = data['min_item_price'] * .01
data['max_item_price'] = data['max_item_price'] * .01

data[['subtotal', 'min_item_price', 'max_item_price']]

,subtotal,min_item_price,max_item_price
0,34.41,5.57,12.39
1,19.00,14.00,14.00
2,19.00,19.00,19.00
3,69.00,6.00,18.00
4,39.00,11.00,16.00
...,...,...,...
197423,13.89,3.45,6.49
197424,30.10,4.05,8.25
197425,18.36,3.00,3.99
197426,11.75,5.35,5.35


### Investigate Missing Values
* With approx 200K rows of data, deliveries with missing values can be eliminated without losing too much information.

In [587]:
data.isnull().sum().sort_values(ascending=False)

total_outstanding_orders                        16262
total_busy_dashers                              16262
total_onshift_dashers                           16262
store_primary_category                           4760
order_protocol                                    995
market_id                                         987
estimated_store_to_consumer_driving_duration      526
total_delivery_seconds                              7
total_delivery_minutes                              7
total_delivery_time                                 7
delivered_at_time                                   7
delivered_at_date                                   7
store_id                                            0
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
created_at_time             

In [588]:
data = data.dropna()

data.isnull().sum().sort_values(ascending=False)

delivery_id                                     0
market_id                                       0
estimated_order_place_duration                  0
total_outstanding_orders                        0
total_busy_dashers                              0
total_onshift_dashers                           0
max_item_price                                  0
min_item_price                                  0
num_distinct_items                              0
subtotal                                        0
total_items                                     0
order_protocol                                  0
store_primary_category                          0
store_id                                        0
total_delivery_minutes                          0
total_delivery_seconds                          0
total_delivery_time                             0
delivered_at_time                               0
created_at_time                                 0
delivered_at_date                               0


## Exploratory Data Analysis

### Create 'Total Delivery' Time Column

In [589]:
deliveries = data.copy()

deliveries

,delivery_id,market_id,created_at_date,delivered_at_date,created_at_time,delivered_at_time,total_delivery_time,total_delivery_seconds,total_delivery_minutes,store_id,...,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,22:24:17,23:27:16,0 days 01:02:59,3779.0,62.98,1845,...,4,34.41,4,5.57,12.39,33.0,14.0,21.0,446,861.0
1,1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,21:49:25,22:56:29,0 days 01:07:04,4024.0,67.07,5477,...,1,19.00,1,14.00,14.00,1.0,2.0,2.0,446,690.0
8,8,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,00:11:35,00:38:01,0 days 00:26:26,1586.0,26.43,5477,...,4,47.71,3,8.20,16.04,8.0,6.0,18.0,446,289.0
14,14,1.0,2015-02-12 03:36:46,2015-02-12 04:14:39,03:36:46,04:14:39,0 days 00:37:53,2273.0,37.88,2841,...,1,15.25,1,15.25,15.25,5.0,6.0,8.0,446,795.0
15,15,1.0,2015-01-27 02:12:36,2015-01-27 03:02:24,02:12:36,03:02:24,0 days 00:49:48,2988.0,49.80,2841,...,2,36.20,2,14.25,21.95,5.0,5.0,7.0,446,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,00:19:41,01:24:48,0 days 01:05:07,3907.0,65.12,2956,...,3,13.89,3,3.45,6.49,17.0,17.0,23.0,251,331.0
197424,197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,00:01:59,00:58:22,0 days 00:56:23,3383.0,56.38,2956,...,6,30.10,4,4.05,8.25,12.0,11.0,14.0,251,915.0
197425,197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,04:46:08,05:36:16,0 days 00:50:08,3008.0,50.13,2956,...,5,18.36,3,3.00,3.99,39.0,41.0,40.0,251,795.0
197426,197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,18:18:15,19:23:22,0 days 01:05:07,3907.0,65.12,3630,...,1,11.75,1,5.35,5.35,7.0,7.0,12.0,446,384.0


## Visualizations

## Model Building

## Conclusion